## 多层感知器实验
* 实验单层感知器XOR问题
* 解决多层感知器XOR问题

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### 创建XOR数据

In [ ]:
datas = np.array([[0,0],[1,1],[0,1],[1,0]])
labels = [1,1,0,0]

### 建立绘图函数

In [ ]:
def plot_fun(datas, labels, w1=0, w2=0, theta=0):
    # 设置坐标系
    plt.xlim((-1,1.5)) # 让坐标系维持在(0,2)范围
    plt.ylim((-1,1.5))
    plt.xlabel('x1') # 为坐标系的两个轴创建名称
    plt.ylabel('x2')
    
    # 画数据点
    A = [] # 存放 0 类型
    B = [] # 存放 1 类型
    for index in range(len(labels)):
        if labels[index] == 0:
            A.append(datas[index])
        else:
            B.append(datas[index])
    A = np.array(A)
    B = np.array(B)
    plt.plot(A[:,0],A[:,1],'r*')
    plt.plot(B[:,0],B[:,1],'bD')
    
    # 如果权值不全是初始值0，就画决策函数线
    if (w1!=0 or w2!=0):
        print('OK')
        x1 = np.linspace(0,1,200)
        if w2 == 0:
            x2 = 0 * x1
        else:
            x2 = (- w1*x1 + theta) / w2
        plt.plot(x1, x2, 'g-', lw=5)
    plt.show()

### 设定超参

In [ ]:
lr = 1.0
epochs = 3

## 单层感知器

### 创建单层感知器模型
* 设定超参数
* 创建模型
* 训练
* 测试

In [ ]:
class perception():
    
    def __init__(self, datas, labels, w1=0, w2=0, theta=0):
        self.datas = datas
        self.labels = labels
        self.w1 = w1
        self.w2 = w2
        self.theta = theta
    
    def active_fun(self, net):
        if net >= 0:
            y = 1
        elif net < 0:
            y = 0
        return y
    
    def model(self, x, y_):
        net = self.w1 * x[0] + self.w2 * x[1] - self.theta
        y = self.active_fun(net)
        if y == 0 and y_ == 1:
            self.w1 = self.w1 + lr * x[0]
            self.w2 = self.w2 + lr * x[1]
            self.theta -= lr
        elif y == 1 and y_ == 0:
            self.w1 = self.w1 - lr * x[0]
            self.w2 = self.w2 - lr * x[1]
            self.theta += lr
        return y, y_
    
    def plot(self):
        return plot_fun(self.datas, self.labels, self.w1, self.w2, self.theta)
            
    def train(self):
        for epoch in range(epochs):
            for index in range(datas.shape[0]):
                y, y_ = self.model(datas[index], labels[index])
                self.plot()
                print("感知器计算结果",y)
                print("正确结果", y_)
                print("w目前更新为：",self.w1, self.w2, self.theta)

### 运行单层感知器模型

In [ ]:
network = perception(datas, labels)
network.train()

## 多层感知器

![mlp](pic/mlp.png)

### 创建多层感知器模型
* A,B为隐藏层神经元，C为输出层神经元
* Sigmoid 激活函数
* 让我们试试 Tensorflow 吧
* Tensorflow 的安装命令： pip install tensorflow

![sigmoid](pic/sigmoid.png)

In [ ]:
import tensorflow as tf
np.set_printoptions(suppress=True) 

datas = np.array([[0.0,0.0],[1.0,1.0],[0.0,1.0],[1.0,0.0]], dtype=np.float32)
labels = np.array([[1.0],[1.0],[0.0],[0.0]], dtype=np.float32)

### 设置参数

In [ ]:
input_num = 2
h1_num = 4
label_num = 1
epoch = 100

W1 = tf.Variable(tf.truncated_normal([input_num, h1_num]))
W2 = tf.Variable(tf.truncated_normal([h1_num,label_num]))
B1 = tf.Variable(tf.truncated_normal([h1_num]))
B2 = tf.Variable(tf.truncated_normal([label_num]))
x  = tf.placeholder(tf.float32, [None, input_num])
y_ = tf.placeholder(tf.float32, [None, label_num])

### 使用 Tensorflow 编写模型

In [ ]:
# 正向传播
hidden1 = tf.sigmoid(tf.matmul(x, W1) + B1)
logits = tf.sigmoid(tf.matmul(hidden1, W2) + B2)

# 反向传播
loss = tf.losses.absolute_difference(labels=y_, predictions=logits)
train_step = tf.train.AdamOptimizer(0.1).minimize(loss)

### 训练

In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for _ in range(epoch):
        _, output_, loss_, W1_, W2_ = sess.run([train_step, logits, loss, W1, W2], feed_dict={x: datas, y_: labels})
    print(output_, loss_)